In [7]:
%load_ext autoreload
%autoreload 2
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sys
sys.path.append('../..')
sys.path.append('..')
import scripts.get_metadata_thesis_bs4 as get_metadata_thesis_bs4
import scripts.get_url_theses_selenium as get_url_theses_selenium
import scripts.get_metadata_thesis_selenium as get_metadata_thesis_selenium
import scripts.utilities_database as utilities_database
import scripts.RAG as RAG
from bs4 import BeautifulSoup



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/avicenne/anaconda3/envs/ultra/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Reset excel

In [13]:
df = pd.read_excel("./static/excel/df_query.xlsx")
df = df.drop(df.columns, axis=1)
df.to_excel("./static/excel/df_query.xlsx", index=False)

In [11]:
df = pd.read_excel("./static/excel/df_query.xlsx")
[i.strip() for i in df.columns]

['Auteur / Autrice',
 'Direction',
 'Type',
 'Discipline(s)',
 'Date',
 'Etablissement(s)',
 'Ecole(s) doctorale(s)',
 'Partenaire(s) de recherche',
 'Jury',
 'Rapporteurs / Rapporteuses',
 'content',
 'title',
 'Établissement co-accrédité',
 'Examinateurs / Examinatrices',
 'Equipe de recherche',
 'Laboratoire',
 "Etablissement d'accueil",
 'référent',
 "établissement opérateur d'inscription",
 'content_condensed',
 'url_query']

In [123]:
response = requests.get("https://theses.fr/2024UPASL053")
soup     = BeautifulSoup(response.text, 'html.parser')
soup.find_all('tr', {'data-v-276fb210': True})
dict_infos = {}
for i in soup.find_all('tr', {'data-v-276fb210': True}):
    infos = i.text.split(":")
    print(infos)
    dict_infos[infos[0].replace("\xa0", "")] = infos[-1].replace("\xa0", "")
    
print(dict_infos)

['Auteur / Autrice\xa0', '\xa0Zhenrui Pan']
['Direction\xa0', '\xa0Svetlana Dokudovskaya,\xa0Yegor Vassetzky']
['Type\xa0', '\xa0Thèse de doctorat']
['Discipline(s)\xa0', '\xa0Sciences du Cancer']
['Date\xa0', '\xa0Soutenance le  20/09/2024']
['Etablissement(s)\xa0', '\xa0université Paris-Saclay']
['Ecole(s) doctorale(s)\xa0', '\xa0École doctorale Cancérologie ', ' biologie-médecine-santé (Villejuif, Val-de-Marne ; 2015-....)']
['Partenaire(s) de recherche\xa0', '\xa0Laboratoire ', " Aspects métaboliques et systémiques de l'oncogénèse pour de nouvelles approches thérapeutiques (Villejuif, Val-de-Marne ; 2020-....) "]
['référent ', ' Université Paris-Saclay. Faculté de médecine (Le Kremlin-Bicêtre, Val-de-Marne ; 2020-....) ']
['graduate school ', ' Université Paris-Saclay. Graduate School Life Sciences and Health (2020-….) ']
['Jury\xa0', '\xa0Président / Présidente ', ' Alexander Ishchenko']
['Examinateurs / Examinatrices ', ' Eric Julien, Nathalie Mazure, Maria Castedo, Pierre-Emmanu

### Get url theses

In [2]:
list_url, _ = get_url_theses_selenium.get_all_url_theses("water")
print(len(list_url))

100%|██████████| 1700/1700 [00:07<00:00, 231.27it/s]


100


In [4]:
list(set([1,1,1,1,1]))

[1]

In [17]:
list_url, _ = get_url_theses_selenium.get_all_url_theses_parallelize("water")

100%|██████████| 1700/1700 [00:00<00:00, 48564.69it/s]


futures


1700it [00:04, 354.96it/s]


## get metadata

In [16]:
get_metadata_thesis_selenium.get_all_metadata_thesis(list_url)

  2%|▏         | 2/100 [00:02<01:50,  1.13s/it]

div[data-v-c8bc896e] not found


  4%|▍         | 4/100 [00:03<01:02,  1.53it/s]

div[data-v-35e8592d] not found


 11%|█         | 11/100 [00:05<00:30,  2.94it/s]

...

 85%|████████▌ | 85/100 [00:32<00:04,  3.02it/s]

div[data-v-35e8592d] not found


100%|██████████| 100/100 [00:37<00:00,  2.66it/s]

div[data-v-35e8592d] not found


,Auteur / Autrice,Direction,Type,Discipline(s),Date,Etablissement(s),Ecole(s) doctorale(s),Partenaire(s) de recherche,content,title,Jury,Examinateurs / Examinatrices,Rapporteurs / Rapporteuses,Laboratoire,Equipe de recherche,référent,Etablissement d'accueil,Établissement co-accrédité,établissement opérateur d'inscription
0,Pratikshya Joshi,Marion Prudent,Projet de thèse,Sciences agronomiques,Inscription en doctorat le 01/10/2023,Bourgogne Franche-Comté,"École doctorale Environnements, Santé (Dijon...",Laboratoire,Grain legumes are cultivated for their protein...,Vers l'optimisation de l'architecture du systè...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Le but de ce travail est de construire une nou...,Contributions aux schémas préservant des solut...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Thomas Meunier,"Xavier Carton, Desmond Barton",Thèse de doctorat,Océanographie physique,Soutenance en 2011,Brest,NaN,NaN,Cette thèse traite de la dynamique des filamen...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Vrinda Sharma,Karen Macours,Projet de thèse,Sciences economiques,Inscription en doctorat le 23/10/2023,Paris 1,École doctorale d'Économie (Paris ; 2004-....),Laboratoire,Missing Value,Climate change and the economics of water,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Marko Stipaničev,Régine Basséguy,Thèse de doctorat,Génie des procédés et de l'environnement,Soutenance le 27/08/2013,"Toulouse, INPT","École doctorale Mécanique, énergétique, géni...",Laboratoire,"L’objectif de ce projet, réalisé dans le cadre...",Improved decision support within biocorrosion ...,Président / Présidente,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Frédéric Bosseur,Pierre Orenga,Thèse de doctorat,Mathématiques,Soutenance en 1998,Corte,NaN,NaN,L'objectif de ce travail est l'étude des écoul...,Détermination de conditions aux limites en mer...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,Alexandre Paris,"Stéphane Abadie, Philippe Heinrich",Thèse de doctorat,Géosciences,Soutenance le 13/09/2021,Pau,École doctorale sciences exactes et leurs ap...,Laboratoire,Les effondrements gravitaires sont la deuxième...,Comparaison de modèles de tsunamis générés par...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,Paul Moal-Darrigade,"Emmanuelle Ducassou, Jacques Giraudeau",Thèse de doctorat,Sédimentologie marine et paléoclimats,Soutenance le 18/03/2022,Bordeaux,École doctorale Sciences et Environnements (...,Laboratoire,La sédimentation de la pente moyenne du Golfe ...,Dynamique de la Mediterranean Outflow Water da...,Président / Présidente,"Gemma Ercilla, Adriano R. Viana",Francisco Javier Sierro,NaN,NaN,NaN,NaN,NaN,NaN
98,Marc Pavec,Xavier Carton,Thèse de doctorat,Océanographie physique,Soutenance en 2007,Brest,NaN,NaN,Les écoulements côtiers sont soumis à diverses...,Instabilités barocline et paramétrique des cou...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Liste d'URLs à scrapper
list_of_urls = [
    'https://example.com',  # Remplace par tes propres URLs
    'https://example.org'
]

# Créer une instance de CrawlerProcess pour exécuter le spider sans un projet Scrapy complet
process = CrawlerProcess(settings={
    "FEEDS": {
        "output.json": {"format": "json"},  # Enregistrer les résultats dans un fichier JSON
    },
})

# Démarrer le processus de scraping avec la liste d'URLs
process.crawl(MySpider, urls=list_url)
process.start()  # Le script attend que le spider ait terminé

2024-10-05 17:06:10 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-10-05 17:06:10 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.12.9, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.7.0, Python 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:35:20) [Clang 16.0.6 ], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.1, Platform macOS-15.0-arm64-arm-64bit
2024-10-05 17:06:10 [scrapy.addons] INFO: Enabled addons:
[]
2024-10-05 17:06:10 [py.warnings] WARNING: /Users/avicenne/anaconda3/envs/ultra/lib/python3.12/site-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See t

In [3]:
get_metadata_theses_beautiful_soup.get_all_metadata_theses_bs_parallelized(list_url)

100%|██████████| 100/100 [00:00<00:00, 8415.37it/s]


futures


74it [02:12,  1.79s/it]


In [7]:
df_metadata = get_metadata_theses_beautiful_soup.get_all_metadata_theses_bs(list_url)

100%|██████████| 100/100 [01:07<00:00,  1.48it/s]


In [8]:
df_metadata.head(2)

,Auteur / Autrice,Direction,Type,Discipline(s),Date,Etablissement(s),Ecole(s) doctorale(s),Partenaire(s) de recherche,title,content,...,Jury,Examinateurs / Examinatrices,Rapporteurs / Rapporteuses,Equipe de recherche,Établissement co-accrédité,Laboratoire,Etablissement d'accueil,établissement opérateur d'inscription,référent,graduate school
0,Lucia Benavente,"Pierre Aimar,Patrice Bacchin",Thèse de doctorat,Génie des procédés et de l'environnement,Soutenance le 03/11/2016,Toulouse 3,"École doctorale Mécanique, énergétique, génie ...",Laboratoire de génie chimique (Toulouse ; 199...,Low fouling membranes for water and bio tech a...,La pénurie d'eau est devenue un des problèmes ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Marc Duretz,André Rist,Thèse de doctorat,Métallurgie,Soutenance en 1989,"Châtenay-Malabry, Ecole centrale de Paris",NaN,NaN,La combustion dans le four water-jacket à plom...,Les performances du haut fourneau à plomb wate...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
df_metadata.to_dict(orient= "records")

[{'Auteur / Autrice': 'Nicole Haroun',
  'Direction': 'Jean-François Landrier,Lourdes Mounien',
  'Type': 'Thèse de doctorat',
  'Discipline(s)': 'Biologie santé. Pathologie vasculaire et nutrition',
  'Date': 'Soutenance le  22/07/2022',
  'Etablissement(s)': 'Aix-Marseille',
  'Ecole(s) doctorale(s)': 'École Doctorale Sciences de la Vie et de la Santé (Marseille)',
  'Partenaire(s) de recherche': ' Centre de recherche en cardiovasculaire et nutrition (C2VN)  - Laboratoire Nutriments Lipides et Prévention des Maladies Métaboliques (Marseille) ',
  'Jury': ' Patrice Darmon',
  'Examinateurs / Examinatrices ': ' Christine Feillet-Coudray',
  'Rapporteurs / Rapporteuses ': ' Jérémie Talvas, Catherine Riva-Lavieille',
  'title': "Carence maternelle en vitamine D chez la souris : conséquences sur l'inflammation du tissu adipeux blanc et la stéatose hépatique dans la descendance",
  'content': "La VD joue un rôle crucial dans le développement périnatal. Mais la VDD est devenue un problème d

In [147]:
utilities_database.update_excel_query_search(pd.DataFrame(), df_metadata, "example.fr")

,Auteur / Autrice,Direction,Type,Discipline(s),Date,Etablissement(s),Ecole(s) doctorale(s),Partenaire(s) de recherche,Jury,Examinateurs / Examinatrices,Rapporteurs / Rapporteuses,title,content,url_these,référent,Établissement co-accrédité,url_query
0,Nicole Haroun,"Jean-François Landrier,Lourdes Mounien",Thèse de doctorat,Biologie santé. Pathologie vasculaire et nutri...,Soutenance le 22/07/2022,Aix-Marseille,École Doctorale Sciences de la Vie et de la Sa...,Centre de recherche en cardiovasculaire et nu...,Patrice Darmon,Christine Feillet-Coudray,"Jérémie Talvas, Catherine Riva-Lavieille",Carence maternelle en vitamine D chez la souri...,La VD joue un rôle crucial dans le développeme...,https://theses.fr/2022AIXM0319,Missing value,Missing value,example.fr
1,Shuaishuai Deng,Benjamin Loubet,Projet de thèse,Géosciences,Inscription en doctorat le 01/10/2023,université Paris-Saclay,École doctorale Sciences de l'environnement d'...,ECOSYS Écologie fonctionnelle et écotoxicolog...,Missing value,Missing value,Missing value,Neglected impacts of air pollution and foliar ...,"Aerosols are deposited on leaf surfaces, which...",https://theses.fr/s367373,Université de Versailles-Saint-Quentin-en-Yve...,Missing value,example.fr
2,Sutha Leenawat,Flora Blanchon,Thèse de doctorat,Histoire de l'art et archéologie,Soutenance en 2001,Paris 4,Missing value,Missing value,Missing value,Missing value,Missing value,La peinture murale de l'école de Bangkok penda...,"Au XIX ième siècle, la Thai͏̈lande dut faire f...",https://theses.fr/2001PA040146,Missing value,Missing value,example.fr
3,Pierre Morel,"Pierre Bertrand,Etienne Gravier",Thèse de doctorat,Physique,Soutenance le 04/07/2008,Nancy 1,EMMA - Ecole Doctorale Energie - Mécanique - M...,LPMIA - Laboratoire de Physique des Milieux I...,Alain Ghizzo,"Dominique Escande, Giovanni Manfredi, Xavier ...",Missing value,Le modèle « water bag » appliqué aux équations...,Ce travail a porté sur l'étude des instabilité...,https://theses.fr/2008NAN10153,Missing value,Missing value,example.fr
4,Arunabha Kundu,Henri Delmas,Thèse de doctorat,Génie des procédés et environnement,Soutenance en 2004,"Toulouse, INPT",Missing value,Missing value,Missing value,Missing value,Missing value,"Réacteurs à trois phases : A, hydrodynamique e...",Two investigations on three phase reactors are...,https://theses.fr/2004INPT003G,Missing value,Missing value,example.fr
5,Missing value,Missing value,Missing value,Missing value,Missing value,Missing value,Missing value,Missing value,Missing value,Missing value,Missing value,Missing value,Missing value,https://theses.fr/2016PA066125,Missing value,Missing value,example.fr
6,Laura Gouriou,Géraldine Gouhier,Projet de thèse,Chimie,Inscription en doctorat le 28/09/2022,Normandie,École doctorale normande de chimie (Caen),"réactivité et analyse (Mont-Saint-Aignan, Sei...",Missing value,Missing value,Missing value,New Ferromagnetic nanoparticles for water reme...,Missing value,https://theses.fr/s392384,Missing value,Université de Rouen Normandie (1966-....),example.fr
7,François Joseph Chatelon,Pierre Orenga,Thèse de doctorat,Mathématiques,Soutenance en 1996,Corte,Missing value,Missing value,Missing value,Missing value,Missing value,Analyse d'un problème de shallow water,Ce mémoire est consacré à l'étude d'un problèm...,https://theses.fr/1996CORT3018,Missing value,Missing value,example.fr
8,Randall Cherry,Geneviève Fabre,Thèse de doctorat,"Lettres, sciences humaines et sociales",Soutenance en 2002,Paris 7,Missing value,Missing value,Missing value,Missing value,Missing value,Mythologie de la voix noire : Ethel Waters et ...,Ce travail étudie la façon dont la chanteuse n...,https://theses.fr/2002PA070064,Missing value,Missing value,example.fr
9,Gilles Darblade,Alain-Yves Leroux,Thèse de doctorat,Mathématiques appliquées,Soutenance en 1997,Bordeaux 1,Missing value,Missing value,Missing value,Missing value,Missing value,Méthodes numériques et conditions aux limites ...,Les modeles shallow water multi

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

EMBEDDING = HuggingFaceEmbeddings(model_name= "all-MiniLM-L6-v2")
db = utilities_database.load_vector_store("./static/vector_store", embeddings=EMBEDDING)

In [10]:
RAG.document_into_context(db.similarity_search("test 1", 10))


' - Title document_0 : Missing value - Document_0 content : Missing value\n\n - Title document_1 : Synthèse et caractérisation de membranes hybrides pour la conduction des ions lithium, et application dans les batteries lithium-air à électrolyte aqueux - Document_1 content : La technologie lithium-air à électrolyte aqueux pourrait révolutionner le stockage de l\'énergie, mais la protection du lithium métallique par une vitrocéramique conductrice du lithium reste une limitation importante. Cela rend le système plus fragile, limite sa cyclabilité et augmente la chute ohmique. L\'objectif de ce travail a été de remplacer cette vitrocéramique par une membrane hybride réalisée par extrusion électro assistée ou electrospinning, qui combine des propriétés d\'étanchéité à l\'eau, de flexibilité et de conductivité du lithium. La conductivité ionique est apportée par la partie céramique, pour laquelle les matériaux Li1,4Al0,4Ti1,6(PO4)3 (LATP) et Li0,33La0,57TiO3 (LLTO) ont été étudiés. L\'étanc

In [22]:
import scripts.get_metadata as get_metadata


get_metadata.get_metadata_theses("https://theses.fr/2016PA066125")


{'Auteur / Autrice : ': 'Tomasz Bobinski', 'Direction : ': 'Agnès Maurel', 'Type : ': 'Philippe Petitjeans', 'Discipline(s) : ': 'Vincent Pagneux', 'Date : ': 'Paris 6', 'Etablissement(s) : ': 'École doctorale Sciences mécaniques, acoustique, électronique et robotique de Paris (2000-....)', 'Paris 6': 'Physique et mécanique des milieux hétérogènes (Paris ; 1997-....)', 'Ecole(s) doctorale(s) : ': 'Christophe Josserand,', 'École doctorale Sciences mécaniques, acoustique, électronique et robotique de Paris (2000-....)': 'Jacek Rokicki', 'Partenaire(s) de recherche : ': 'Sébastien Guenneau,', 'Laboratoire :': 'Richard Porter'}


{'title': 'Metamaterials for water waves',
 'content': 'Cette thèse porte sur l’étude numérique et expérimentale de l’utilisation de métamatériaux pour le contrôle des ondes à la surface de l’eau. Dans la première partie, nous avons montré comment focaliser les ondes à la surface de l’eau en utilisant une analogie existant avec des métamatériaux électromagnétiques de permittivité diélectrique quasi nulle qualifiés de ”epsilon-near-zero”. Cela a permis d’adapter le motif de phase à l’interface circulaire entre deux domaines présentant un contraste élevé de profondeur d’eau. L’analogie a donné lieu a un comportement fortement non linéaire des ondes, qui se manifeste par la génération d’une cascade de points focaux associés à des sous-multiples de longueurs d’onde par rapport à l’onde incidente. La deuxième application envisagée des métamatériaux pour les ondes à la surface est de rendre invisible les défauts géométriques d’un guide pour un observateur situé en champ lointain. Dans le pre